In [98]:
import pandas as pd
import numpy as np

In [51]:
games = pd.read_csv('nfl-football-player-stats/games_1995.csv')
players = pd.read_csv('nfl-football-player-stats/players_1995.csv')

In [258]:
games.loc[0,]

age                                    28-114
date                               1995-09-03
defense_interception_touchdowns             0
defense_interception_yards                  0
defense_interceptions                       0
defense_sacks                               0
defense_safeties                            0
defense_tackle_assists                      0
defense_tackles                             0
field_goal_attempts                         0
field_goal_makes                            0
game_location                               H
game_number                                 1
game_won                                False
kick_return_attempts                        0
kick_return_touchdowns                      0
kick_return_yards                           0
opponent                                  KAN
opponent_score                             34
passing_attempts                            0
passing_completions                         0
passing_interceptions             

In [266]:
half_ppr = {
    'rushing_yards': 0.1,
    'rushing_touchdowns': 6,
    'receiving_receptions': 0.5,
    'receiving_yards': 0.1,
    'receiving_touchdowns': 6
}
def get_ROS_stats(games, players, player_ids, position, year, nfl_week, points_dict):
    # Returns pandas Dataframe with columns player_id, name, year, nfl_week, ROS_ppg, ROS_games
    for_df = []
    for p_id in player_ids:  
        ROS = games[(games.player_id == p_id) & (games.year == year) & 
                    (games.game_number >= nfl_week) & (games.game_number <= 16)]
        ROS_games = len(ROS)
        ROS_pts = 0
        for stat, value in zip(points_dict.keys(), points_dict.values()):
            ROS_pts += sum(ROS[stat])*value
        ROS_ppg = np.round(ROS_pts/ROS_games,2)
        name = players[players['player_id'] == p_id].name.values[0]
        for_df.append([p_id, name, year, nfl_week, ROS_ppg, ROS_games])
    df = pd.DataFrame(for_df, columns =['player_id', 'name', 'year', 'nfl_week', 'ROS_ppg', 'ROS_games'])
    return df

In [267]:
# usage example to get ROS stats for AP and Foster in 2011
df = get_ROS_stats(games, players, [17463, 7173], 'RB', 2011, 5, half_ppr)
df

,player_id,name,year,nfl_week,ROS_ppg,ROS_games
0,17463,Adrian Peterson,2011,5,16.61,8
1,7173,Arian Foster,2011,5,23.05,11
